Прочитать как скачать датасет с каггла - [тут](https://github.com/Kaggle/kaggle-api).

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = ""

In [ ]:
!kaggle datasets download -d slothkong/10-monkey-species

 99%|████████████████████████████████████████▌| 542M/547M [00:04<00:00, 136MB/s]
100%|█████████████████████████████████████████| 547M/547M [00:04<00:00, 131MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile('10-monkey-species.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [1]:
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision
import torchvision.models as models
import pytorch_lightning as pl
import torch.nn as nn
from torchmetrics import Accuracy
import torch.optim as optim
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from collections import Counter
import torch.nn.functional as F

# Dataset

In [2]:
transforms = transforms.Compose([transforms.Resize((224,224)),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])

In [3]:
train_ds = ImageFolder('./data/training/training/',transform=transforms)
test_ds = ImageFolder('./data/validation/validation/',transform=transforms)

In [4]:
train_cnt = Counter(train_ds.targets)
test_cnt = Counter(test_ds.targets)
print(train_cnt)
print(test_cnt)

Counter({3: 122, 7: 114, 5: 113, 1: 111, 2: 110, 6: 106, 8: 106, 0: 105, 4: 105, 9: 105})
Counter({3: 30, 1: 28, 5: 28, 7: 28, 2: 27, 8: 27, 0: 26, 4: 26, 6: 26, 9: 26})


# Dataloader

In [5]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True,num_workers=4)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False,num_workers=4)

# Model

In [6]:
class HWModel(pl.LightningModule):
    def __init__(self, num_classes, lr=2e-4):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.num_classes = num_classes
        self.model = models.resnet50(pretrained=True)
        in_feat = list(self.model.children())[-1].in_features
        self.model.fc = nn.Linear(in_feat, self.num_classes)
        
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.lr)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        
        x, y = batch
        
        preds = self(x)
        loss = self.loss_fn(preds, y)
        acc = self.accuracy(torch.argmax(preds, dim=1), y)
        
        self.log('train_loss', loss.item(), on_epoch=True,logger=True)
        self.log('train_acc', acc, on_epoch=True,logger=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        
        x,y = batch
        
        preds = self(x)
        
        loss = self.loss_fn(preds, y)
        acc = self.accuracy(torch.argmax(preds, dim=1), y)
        
        self.log('val_loss', loss, on_epoch=True,prog_bar=True,logger=True)
        self.log('val_acc', acc, on_epoch=True,prog_bar=True,logger=True)
        
    def test_step(self, batch, batch_idx):
        
        x,y = batch
        preds = self(x)
        acc = self.accuracy(torch.argmax(preds, dim=1), y)
        
        self.log('test_acc', acc, on_epoch=True,prog_bar=True)

# Train

In [7]:
from torch import Tensor
from typing import Type

In [8]:
model = HWModel(num_classes = 10,lr = 1e-3)
trainer_args = {
        "accelerator": "gpu",
        "max_epochs": 10,
    }
trainer = pl.Trainer(**trainer_args,enable_progress_bar=True)
trainer.fit(model,train_dl,test_dl)

/home/dmitryindenbom/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dmitryindenbom/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/dmitryindenbom/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dmitryindenbom/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/doc

Sanity Checking: 0it [00:00, ?it/s]

/home/dmitryindenbom/anaconda3/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [1]:
len(train_dl)

NameError: name 'train_dl' is not defined